In [1]:
from manim import *
from PIL import Image
import jupyter_capture_output

video_scene = " -v WARNING --disable_caching fe_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s fe_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [17]:
# function to turn positions into complex numbers
def pos_to_complex(pos_array):
    N = len(pos_array)
    x_array = np.zeros((N,), dtype = np.complex_)
    for n in range(N):
        x_n_real = pos_array[n][0]
        x_n_imag = pos_array[n][1]
        x_array[n] = complex(x_n_real, x_n_imag)
    return x_array


# discrete fourier transform
def dft(x_array, order):
    N = len(x_array)
    k_array = np.zeros((order,), dtype = np.complex_)
    for k in range(order):
        for n in range(N):
            k_array[k] += x_array[n] * np.exp(-1j * 2*np.pi*k/N*n)
    return k_array

In [ ]:
# reading image and grayscale conversion
image_elephant = Image.open('../external_media/elephant_line_drawing.png').convert('L')
image_elephant_array = np.array(image_elephant)


# image processing
n_height, n_width = image_elephant_array.shape
pixel_list = []
# define image size
y_min = 230
y_max = 230
x_min = 122
x_max = 122

for i_height in range(n_height):
    for i_width in range(n_width):
        if image_elephant_array[i_height, i_width] < 100:
            pixel_list.append(np.array([i_width, i_height]))
            if i_width > x_max:
                x_max = i_width
            if i_width < x_min:
                x_min = i_width
            if i_height > y_max:
                y_max = i_height
            if i_height < y_min:
                y_min = i_height


# fit the image to manim
x_range = x_max - x_min
y_range = y_max - y_min

x_manim_range = 14
y_manim_range = 8

x_manim_offset = 0.5
y_manim_offset = 0.5

n_points = len(pixel_list)

print(f"Number Points: {n_points}")
print(f"x-range: {x_min}, {x_max}")
print(f"y-range: {y_min}, {y_max}")



# transform pixel list to manim coordinates
coordinate_list = []
for x, y in pixel_list:
    # print(float(x), float(y))
    x_point = (float(x) - x_min) / x_range * (x_manim_range-2*x_manim_offset) + x_manim_offset - x_manim_range / 2
    y_point = (float(y) - y_min) / y_range * (y_manim_range-2*y_manim_offset) + y_manim_offset - y_manim_range / 2
    coordinate_list.append((x_point, -y_point, 0))



# function to calculate the distance between two points
def get_distance(pos1, pos2):
    return np.sqrt( (pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2 + (pos1[2]-pos2[2])**2 )



# function to insert a coordinate into a list into an appropriate position, return values: True (coordinate becomes new reference), False (standard)
def insert_coordinate(input_coordinate, coordinate_list):

    # initial guess for closest neighbor
    min_dist = get_distance(input_coordinate, coordinate_list[0])
    min_dist_index = 0

    # iterating though list to find actual closest neighbor
    for i, coordinate in enumerate(coordinate_list):
        dist = get_distance(input_coordinate, coordinate)
        # replace closest neighbor for even closer neighbor
        if dist < min_dist:
            min_dist = dist
            min_dist_index = i

    # append new coordinate at the end (set input variable as reference: True)
    if min_dist_index == len(coordinate_list)-1:
        coordinate_list.append(input_coordinate)
        return True
    # insert new coordinata into list (reference remains untouched: False)
    else:
        pre_dist = get_distance(input_coordinate, coordinate_list[min_dist_index-1])
        post_dist = get_distance(input_coordinate, coordinate_list[min_dist_index+1])
        if pre_dist < post_dist:
            coordinate_list.insert(min_dist_index, input_coordinate)
        else:
            coordinate_list.insert(min_dist_index+1, input_coordinate)
        return False



# function to sort a list of coordinates (start is bottom left)
def sort_coordinate_list(coordinate_list, max_dist = x_manim_range):
    copy_coordinate_list = coordinate_list.copy()
    ordered_coordinate_list = []
    reference_coordinate = np.array([-x_manim_range/2, -y_manim_range/2, 0])

    # ordering list items until list is empfty
    while (len(copy_coordinate_list)):
        # initial guess for closest neighbor
        min_dist = get_distance(copy_coordinate_list[0], reference_coordinate)
        min_dist_index = 0
        min_dist_coordinate = copy_coordinate_list[0]

        # iterating though list to find actual closest neighbor
        for i, coordinate in enumerate(copy_coordinate_list):
            dist = get_distance(coordinate, reference_coordinate)
            # replace closest neighbor for even closer neighbor
            if dist < min_dist:
                min_dist = dist
                min_dist_index = i
                min_dist_coordinate = coordinate

        # maximal distance violation
        if min_dist > max_dist:
            if insert_coordinate(min_dist_coordinate, ordered_coordinate_list):
                reference_coordinate = min_dist_coordinate
            # print(min_dist, max_dist, reference_coordinate)
            # print(len(copy_coordinate_list))
            # return ordered_coordinate_list
        else:
            ordered_coordinate_list.append(min_dist_coordinate)
            reference_coordinate = min_dist_coordinate
        # setting up the next iteration 
        copy_coordinate_list.pop(min_dist_index)
    return ordered_coordinate_list



# coordinate_list, 
ordered_coordinate_list_unrefined = sort_coordinate_list(coordinate_list) 
ordered_coordinate_list = sort_coordinate_list(coordinate_list, 3) 
len(ordered_coordinate_list_unrefined), len(ordered_coordinate_list)

Number Points: 3164
x-range: 32, 581
y-range: 121, 316
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


(3164, 3164)

In [73]:
# class to draw the fourier arm for a given frequency array
class DrawComplexFourier(Mobject):
    def __init__(self, center, k_array, N, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.N = N
        self.k_array = k_array
        self.k_max = len(k_array)


    # get absolute of a complex number c
    def get_absolute(self, c):
        return np.sqrt(c.real**2 + c.imag**2)


    # gets the n-th point of the delivered frequency array
    def get_fourier_circles(self, n):
        fourier_circles_group = VGroup()
        position = np.array([*self.center])                                         # current postition
        # iterate through the individual frequencies
        for k in range(self.k_max):
            x_k = self.k_array[k]                                                   # get the cuurent complex amplitude
            x_k_abs = self.get_absolute(x_k) / self.N                               # get the current frequency amplitude

            x_n = x_k / self.N * np.exp(1j * 2*np.pi *k/self.N*n)                   # get the current complex value
            x_n_real = x_n.real                                                     # get the real part of x_n
            x_n_imag = x_n.imag                                                     # get the imaginary part of x_n

            # geometrical manim objects
            circle = Circle(radius = x_k_abs, color = WHITE, stroke_width = 0.25).move_to(position)
            arrow = Arrow(start = position, end = position + np.array([x_n_real, x_n_imag, 0]), color = WHITE, stroke_width = 2-x_k_abs*2/(self.k_max+2), buff = 0)
            fourier_circles_group.add(circle, arrow)

            # update position
            position[0] += x_n_real
            position[1] += x_n_imag
            # fourier_circles_group.position(position)
        self.add(Dot(point = position, color = RED, radius = 0.015))
                                                              
        return fourier_circles_group

In [85]:
%%capture_video --path "../animations/fourier_elephant/fourier_elephant_HD.mp4"
%%manim -qh --fps 60 $video_scene


class fe_Scene(ThreeDScene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        center = np.array([0.0, 0, 0])

        # fourier parameters
        data_line_list = ordered_coordinate_list
        periodicity_N = len(data_line_list)
        # fourier_order = 30
        fourier_order = periodicity_N


        # positions list
        pos_list = [
            (-1, -1, 0),
            (0, -1, 0),
            (1, 0, 0),
            (2, 2, 0),
            (-7, 4, 0),
            (-6, 3, 0)
        ]


        # draw pos list
        def draw_pos_list(pos_list):
            len_pos_list = len(pos_list)
            for i in range(len_pos_list):
                self.add(Dot(point = pos_list[i], color = RED, radius = 0.015))
                self.wait(0.02)

    
        # made up list
        # draw_pos_list(pos_list)                     # draws all the points of the list
        # x_array = pos_to_complex(pos_list)          # transform positions to complex numbers
        # k_array = dft(x_array)                      # dft the complex numbers to the fourier space


        # possible elephant ?
        # draw_pos_list(ordered_coordinate_list)
        x_array = pos_to_complex(data_line_list)
        k_array = dft(x_array, fourier_order)
        
        fourier = DrawComplexFourier(center, k_array, periodicity_N)
        self.add(fourier)

        fourier_circles = fourier.get_fourier_circles(100)
        self.add(fourier_circles)

        self.wait(1.5)
        for i in range(1000, 2000):
            self.remove(fourier_circles)
            fourier_circles = fourier.get_fourier_circles(i)
            self.add(fourier_circles)
            self.wait(0.02)

        self.wait(5)

Output saved by overwring previous file at ../animations/fourier_elephant/fourier_elephant_HD.mp4.


KeyboardInterrupt: 